In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,75054.99,-0.98,-0.19,-0.10,0.00,-0.01,0.00,-0.00
3,-0.98,9622.04,0.08,-0.04,0.01,-0.00,-0.00,0.00
4,-0.19,0.08,3500.63,-0.01,0.03,0.01,-0.00,-0.00
5,-0.10,-0.04,-0.01,805.14,-0.00,-0.00,0.00,0.00
6,0.00,0.01,0.03,-0.00,157.12,0.00,0.00,0.00
7,-0.01,-0.00,0.01,-0.00,0.00,27.19,-0.00,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,3.94,0.00
9,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.35


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00004,-0.00001,-0.00001,0.00000,-0.00001,0.00000,-0.00001
3,-0.00004,1.00000,0.00001,-0.00002,0.00001,-0.00001,-0.00001,0.00000
4,-0.00001,0.00001,1.00000,-0.00001,0.00004,0.00004,-0.00000,-0.00002
5,-0.00001,-0.00002,-0.00001,1.00000,-0.00001,-0.00001,0.00001,0.00000
6,0.00000,0.00001,0.00004,-0.00001,1.00000,0.00002,0.00000,0.00003
7,-0.00001,-0.00001,0.00004,-0.00001,0.00002,1.00000,-0.00001,-0.00001
8,0.00000,-0.00001,-0.00000,0.00001,0.00000,-0.00001,1.00000,0.00002
9,-0.00001,0.00000,-0.00002,0.00000,0.00003,-0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.4326879789293974

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.04433241e+08 1.13830218e+07 1.45615343e+06 7.40906808e+04
 2.64731505e+03 7.05254015e+01 1.10968087e+00 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999803,-0.018618,-0.006660,-0.001529,-0.000298,-0.000052,-0.000007,-6.882342e-07
1,0.018088,0.997327,-0.069418,-0.013628,-0.002637,-0.000457,-0.000066,-5.855531e-06
2,0.007835,0.068507,0.996349,-0.049495,-0.009057,-0.001562,-0.000227,-2.034804e-05
3,0.002141,0.016785,0.047897,0.997366,-0.051074,-0.008429,-0.001218,-1.084115e-04
4,0.000520,0.004058,0.011163,0.049929,0.996864,-0.059654,-0.008301,-7.359565e-04
5,0.000120,0.000934,0.002564,0.011172,0.058357,0.995464,-0.073986,-6.373731e-03
6,0.000026,0.000203,0.000557,0.002421,0.012415,0.072496,0.993285,-8.926336e-02
7,0.000005,0.000036,0.000100,0.000433,0.002217,0.012823,0.088541,9.959874e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00019673160857003325,
 0.0026730296661146014,
 0.003651082487591051,
 0.002634251631009832,
 0.0031360311924933937,
 0.004536463969016635,
 0.006714522713985316,
 0.004012613533145748]